# Session 1: Single Track

# What is **`gos`** 🦆?

## Motivation

So far in the tutorial we have provided a general overview of the Gosling  visualization grammar and introduced its key features:

- _Expressiveness_
- _Data scalability_
- _Encoding scalability_
- _Coordinated interactivity_

Although Gosling is extremely flexible, its JSON represenation is less ergonomic to construct natively via popular programming languages (like Python) and deploying a Gosling-based visualization requires the administration of a web server. Together these challenges serve as a barrier to entry for some, and we were motivated to design a simplified API _for_ computational biologists to visualize their own datasets with Gosling. 

Enter **`gos`**...

## Overview

**`gos`** is a declarative **Python library** designed to create interactive multi-scale visualizations of genomics and epigenomics data. Its main features include:

- Authoring declarative genomics visualizations which adhere to the [Gosling](http://gosling-lang.org/) JSON Specification

- Displaying Gosling visualizations directly in computational notebooks (Jupyter, JupyterLab, Google Colab)

- Transparently hosting genomics datasets for visualizations (hiding web server complexities)

<br>

<p align="center">
  <img width="600" src="https://user-images.githubusercontent.com/24403730/176062140-596b3a74-168f-4b86-8047-85785ee06179.png" />
</p>

## How it works

The **`gos`** Python library exposes a simple API that maps directly to the formal Gosling JSON specification. Users write Python programs with **`gos`** which ultimately:

- Emit JSON (the Gosling visualization)
- Automatically render said JSON within computational notebooks 

<p align="center">
  <img width="600" src="https://pbs.twimg.com/media/FCT0BntX0AEEcDH?format=jpg&name=4096x4096" />
</p>

> **Note** You need not understand the low-level details of **`gos`**. The important thing to keep in mind is that **`gos`** (by design) is _gaurenteed_ to be consistent with the formal Gosling grammar, and therefore learning **`gos`** will teach Gosling and vis versa.

# Getting started

The remainder of this notebook will focus on introducing the Gosling grammar through the declarative Python API.

Start by importing `gosling`.

In [ ]:
!pip install gosling==0.0.8

In [ ]:
import gosling as gos

> **Note** it is a convention to import as `gos` and then access the API through this namespace. 

## Track fundamentals

**`gos`** exposes two fundamental building-blocks for genomics visualizations:

- `gos.Track`
- `gos.View`

A _Track_ is the core component of a genomics visualization that defines explict **transformations** and **mappings** of genomics data to **visual properties**. A _Track_ may be composed with other _Tracks_ or **grouped** into a _View_ that share the same linked genomic domain. 

The figure below depicts the heirarchical structure of a Gosling visualization, displaying **3** distict _Views_ (light orange/blue/green) which individually are composed of several _Tracks_.

<img src="https://github.com/gosling-lang/gos-example/blob/main/notebooks/tracks-views.jpeg?raw=1" width="600">

A _Track_ (dark orange/blue/green) is the base primative for building genomics visualizations and requires binding a data source. In `gos` we define an abstract genomic data source and bind it to a _Track_ directly through the Python API. 

We will start by loading a BED file containing UCSC hg38 cytoband information

In [ ]:
data_url = "https://raw.githubusercontent.com/sehilyi/gemini-datasets/master/data/UCSC.HG38.Human.CytoBandIdeogram.bed"

In [ ]:
!curl -s {data_url} | head | column -t

In [ ]:
# The dataset is a BED4+1 file which can be read in Gosling as the CSV datatype
data = gos.csv(
    url=data_url,
    headerNames=['chrom', 'chromStart', 'chromEnd', 'name', 'stain'], # the +1 field is stain
    chromosomeField="chrom", # the column containing chrom names
    genomicFields=["chromStart", "chromEnd"], # fields with genomic coordinates
    separator="\t",
)

# bind the data to a track
gos.Track(data)

The _Track_ above is now bound to the genomics data, but the Gosling grammar requires the root of every visualization as a _View_, which may contain one or more _Tracks_. 

In order to complete a Gosling specification for the track in isolation, we use the `gos.Track.view()` method to cast the track within a `gos.View`. In Jupyter or Google Colab, the visualization is automatically rendered in the cell below rather than printing a Python object like above.

In [ ]:
track = gos.Track(data) # create `gos.Track`
view = track.view()     # cast to `gos.View`
gos.Track(data).view()

_Something_ appeared on the screen but our visualization looks empty. What's going on? We haven't declared how to map the dataset to any visual properties! 


Use the `gos.Track.mark_*()` and `gos.Track.encode()` methods to specify a **mark** and what **visual encodings** to apply.

In [ ]:
# explore available marks with `gos.Track(data).mark_*`
gos.Track(data).mark_point().encode(
    x=gos.X("chromStart", type="genomic"),
    y=gos.Y("chromStart", type="genomic", axis="left"),
).view()

In [ ]:
gos.Track(data).mark_point().encode(
    x=gos.X("chromStart", type="genomic"),
    y=gos.Y("chromStart", type="genomic", axis="left"), # add y-encoding
).view()

You can read more about the specific visual channels which are supported by each mark type in the [Gosling documentation](http://gosling-lang.org/docs/visual-channel).

## Data-types

The specifics of an encoding depend on the type of the data. Gosling recognizes three datatypes:

| Data Type | Shorthand Code | Description |
| --------- | -------------- | ----------- | 
| quantitative | Q | a continuous real-valued quantity | 
| nominal | N | a discrete unordered category |
| genomic | G | a genomic base-pair position |

Data types can either be expressed in long-form like above, or a short-hand syntax can be used to remove boilerplate when exploring visual encodings.

In [ ]:
gos.Track(data).mark_point().encode(
    x=gos.X("chromStart:G"), # `gos.X("chromStart:G")` also ok and useful to further refine encoding
    y=gos.Y("strain:N")
).view()

In [ ]:
gos.Track(data).mark_point().encode(
    x=gos.X("chromStart:G"),
    y=gos.Y("chromStart:Q", axis="left"), # change y-encoding to quantitative
).view()

In [ ]:
gos.Track(data).mark_point().encode(
    x=gos.X("chromStart:G"),
    y=gos.Y("stain:N") # change y-encoding to nominal field
).view()

In [ ]:
gos.Track(data).mark_point().encode(
    x=gos.X("chromStart:G"),
    y=gos.Y("stain:N"),
    color=gos.value("black"), # no data-type, just a literal value!
).view()

## Multiple encodings

We can use both `y` position and `color` to encoding the same field.

In [ ]:
gos.Track(data).mark_rect().encode(
    x=gos.X("chromStart:G"),
    xe=gos.Xe("chromEnd:G"),
    y=gos.Y("stain:N"),
).view()

In [ ]:
gos.Track(data).mark_point().encode(
    x=gos.X("chromStart:G"),
    y=gos.Y("stain:N"),
    # custom colormapping
    color=gos.Color(
        "stain:N", 
        domain=["gneg", "gpos25", "gpos50", "gpos75", "gpos100", "gvar"],
        range=["white", "#D9D9D9", "#979797", "#636363", "black", "#A0A0F2"],
        legend=True,
    ),
).view()

In [ ]:
# change mark and add text-encoding
gos.Track(data).mark_text().encode(
    x=gos.X("chromStart:G"),
    y=gos.Y("stain:N"),
    color=gos.Color(
        "stain:N", 
        domain=["gneg", "gpos25", "gpos50", "gpos75", "gpos100", "gvar"],
        range=["white", "#D9D9D9", "#979797", "#636363", "black", "#A0A0F2"],
        legend=True,
    ),
    text=gos.Text("stain:N") 
).view()

## Example: Simplified ideogram

We can create a simplified ideogram using the concepts from above by changing the `mark` for our track and specifying additional encodings.

In [ ]:
track = gos.Track(data).mark_rect().encode(
    # defines start and end of rectangle mark
    x=gos.X("chromStart:G", axis="top"),
    xe=gos.Xe("chromEnd:G"),
    # defines how to map Giemsa-stain factor to colors
    color=gos.Color(
        "stain:N", 
        domain=["gneg", "gpos25", "gpos50", "gpos75", "gpos100", "gvar"],
        range=["white", "#D9D9D9", "#979797", "#636363", "black", "#A0A0F2"]
    ),
    # customize the style of the visual marks. 
    size=gos.value(20),
    stroke=gos.value("gray"),
    strokeWidth=gos.value(0.5)
)

track.view()

Additional parameters for the resulting `gos.View` can be passed in as well for convenience. We can easily set a `title` and `xDomain` for our visualization, initializing the initial genomic region to display human chromsome 1. 

In [ ]:
track.view(
    title="Gos is awesome!",
    xDomain=gos.GenomicDomain(chromosome="chr1"),
)

In [ ]:
view = track.view()
view.to_json()

# Track reuse and composition

This section demonstrates how to build more complex, layered tracks and write functions to reuse visualizations for other data sources.

A common pattern in **`gos`** is the reuse of `gos.Track` instances to create new, modified `gos.Track` or `gos.View` objects. This feature allows users to be much more concise with the Python API compared to the Gosling JSON equivalent. 

> **Note** in the code example above, we reuse `track` to create a _new_ view (with `title` and `xDomain`) without redefining the visualization from scratch.

## The data

In this section we will visualize several scATAC-seq "pseudobulk" tracks from [Corces et. al (_Nature Genetics, 2020_)](https://www.nature.com/articles/s41588-020-00721-x) multi-omic atlas of the human brain. Each scATAC-seq track is stored in a separate BigWig file and represents the normalized aggregate signal of all cells from a given cell-type in the human brain.

In [ ]:
urls = [
    f"https://s3.amazonaws.com/gosling-lang.org/data/{file}"
    for file in [
        "ExcitatoryNeurons-insertions_bin100_RIPnorm.bw",
        "InhibitoryNeurons-insertions_bin100_RIPnorm.bw",
        "Microglia-insertions_bin100_RIPnorm.bw",
        "Astrocytes-insertions_bin100_RIPnorm.bw",
    ]
]
urls

In the previous section we visualized a text-based data format which was already tabular (BED). For non-tabular data formats supported by Gosling (e.g., BAM, BigWig) we need to be explicit about how to translate these files to a tabular representation in Gosling for which we can build our visualization. 

In [ ]:
data = gos.bigwig(urls[0], column="position", value="peak")
data

In [ ]:
gos.Track(data).mark_bar().encode(
    x=gos.X("position:G"),
    y=gos.Y("peak:Q", axis="right"),
).view()

## Overlay track

With the quantitative data type for `peak`, we can experiment with different marks in our visualization. Rather than repeating the definition, we use a `base` track to derive other modified tracks.

In [ ]:
base = gos.Track(data).encode(
    x=gos.X("position:G")
).properties(height=60)

heatmap = base.mark_rect().encode(
    color=gos.Color("peak:Q"),
)

heatmap.view()

In [ ]:
line = base.mark_line().encode(
    y=gos.Y("peak:Q"),
    color=gos.value("gray"),
)
line.view()

In [ ]:
points = line.mark_point()
points.view()

In [ ]:
colored_points = points.encode(
    color=gos.Color("peak:Q"), # overrides color encoding
    size=gos.Size("peak:Q"),
)
colored_points.view()

Since the tracks above share some of the same encodings, we can layer the tracks together to create a composite `"overlay"` which combines different marks.

In [ ]:
gos.overlay(line, colored_points) # returns a View

While this example is fairly contrived, the ability to create overlay tracks is an essential feature in Gosling and allows much more complex visual encodings (e.g. genome annotations).

## Track function

The previous section demonstrated further how track definitions may be reused and extended. In this section we will show how a Python function can be defined to reuse a track definition for other data sources.

We can refactor this snippet from earlier so that we have a _function_ that generates a barplot for any scATAC-seq track above.

In [ ]:
def barplot(url: str, title: str = None, color: str = None) -> gos.Track:
    data = gos.bigwig(url, column="position", value="peak")
    track = gos.Track(data).mark_bar().encode(
        x=gos.X("position:G"),
        y=gos.Y("peak:Q", axis="right"),
    )
    if color:
        track = track.encode(color=gos.value(color))
    if title:
        track = track.properties(title=title)
    return track.properties(height=40)

barplot(urls[0]).view()

In [ ]:
urls

We can then reuse this utility to create multiple tracks to combine together.

In [ ]:
tracks = []
for url, color in zip(urls, ["#F29B67", "#3DC491", "#565C8B",  "#77C0FA"]):
    title = url.split("/")[-1].split("-")[0]
    track = barplot(url=url, title=title, color=color)
    tracks.append(track)
tracks

In [ ]:
tracks[1].view()

Combined together into a single visualization with `gos.stack`.

In [ ]:
# returns a `View` which shared genomic domain for all child tracks
gos.stack(*tracks).properties(
    xDomain=gos.GenomicDomain(chromosome="3", interval=[52168000, 52890000]),
)

In [ ]:
gos.stack(*tracks).properties(layout="circular")